In [ ]:
from functools import partial
from collections import namedtuple, Counter
import operator
import os
import secrets
import subprocess
import shutil
import tempfile

import nbconvert
import nbformat
from IPython import display
import ipywidgets
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox,\
                       Textarea, Image, Label, HTML, Dropdown,\
                       Tab, IntText, Select, Checkbox

import traitlets
import pandas
import jinja2
from pony import orm

import db
import emailing_feedback
from app_model import AppModel
from progressbar import ProgressWithTitle

In [ ]:
state = AppModel()

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 2,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

default_email_template = """Dear {{student.first_name.split(' ') | first }} {{student.last_name}},

Below please find attached the scans of your exam and our feedback.
If you have any questions, don't hesitate to contant us.

{% for problem in results | sort(attribute='name') if problem.feedback  -%}
{{problem.name}} (your score: {{problem.score}} out of {{problem.max_score}}):
{% for feedback in problem.feedback %}
    * {{ (feedback.description or feedback.short) | wordwrap | indent(width=6) }}
{% endfor %}
{%- if problem.remarks %}
    * {{ problem.remarks | wordwrap | indent(width=6) }}
{% endif %}
{% endfor %}

{%- if student.total < 60 -%}
Your grade is below 6
{%- elif student.total < 80 -%}
Your grade is between 6 and 8.
{%- else -%}
Your grade is above 8. Very good
{%- endif%}

Best regards,
Solid state course team."""

In [ ]:
%%HTML
<style>
.widget-hbox .widget-label {
    max-width: unset ;
}
</style>

# Widget definitions and interaction

In [ ]:
## Solution selection (submission x problem) matrix
prev_stud = Button(icon='fa-step-backward', description=' previous')
next_stud = Button(icon='fa-step-forward', description=' next')
next_ungraded_stud = Button(icon='fa-forward', description=' next ungraded')
jump_to_stud = Text(description='Jump to', placeholder='student # or name')
traitlets.directional_link((state, 'student'), (jump_to_stud, 'value'))

jump_to_exam = Dropdown(description='Exam', placeholder='exam #')

traitlets.link((state, 'exams'), (jump_to_exam, 'options'))
traitlets.link((state, 'exam_id'), (jump_to_exam, 'value'))

jump_to_problem = Dropdown(description='Jump to', placeholder='problem #')

traitlets.link((state, 'problems'), (jump_to_problem, 'options'))
traitlets.link((state, 'problem_id'), (jump_to_problem, 'value'))

## Solution and FeedbackOption
options_feedback = SelectMultiple(name='feedback')
options_feedback.layout.width = 'flex-stretch'
options_feedback.layout.height = 'flex-stretch'

traitlets.directional_link((state, 'feedback_options'), (options_feedback, 'options'))

remarks = Textarea(placeholder='Student-specific feedback')
remarks.layout.width = '40%'
remarks.layout.height = '300px'
remarks.layout.min_height = '300px'
remarks.layout.max_height = '300px'

traitlets.link((state, 'remarks'), (remarks, 'value'))

full_page = Checkbox(description='Show full page')
traitlets.link((state, 'show_full_page'), (full_page, 'value'))

answer = Image(format='png', width='100%')

## info
graded_label = HTML()
graded_label.template = '<h5>Graded by: {} @ {:%d/%m %H:%M}</h5>'.format

problem_progress = ProgressWithTitle(
    value=0,
    min=0,
    step=1,
    title='Progress',
    bar_style='success',
    orientation='horizontal',
)

traitlets.directional_link((state, 'exam_id'), (problem_progress, 'max'), state.num_submissions)

## grader selection
grader = Dropdown(description='Grader', placeholder='Grader name', options=state.graders)
widgets_to_disable = (options_feedback, remarks)
for w in widgets_to_disable:
    traitlets.directional_link((state, 'grader_id'), (w, 'disabled'), transform=lambda x: x == 0)

option_names = Select(description='Feedback options', options=[('ADD NEW', 0)])
option_names.layout.height = '300px'

traitlets.directional_link((state, 'feedback_options'), (option_names, 'options'),
                           transform=lambda x: [('ADD NEW', 0)] + x)

traitlets.link((option_names, 'value'), (state, 'edited_feedback_option'))

traitlets.directional_link((option_names, 'value'), (state, 'edited_feedback_option'))

option_name = Text(placeholder='Option name')
traitlets.link((state, 'edited_feedback_name'), (option_name, 'value'))

option_score = IntText(description='Score')
traitlets.link((option_score, 'value'), (state, 'edited_score'))

option_description = Textarea(placeholder='Full description')
option_description.layout.width = '100%'

traitlets.link((option_description, 'value'), (state, 'edited_description'))

save_option = Button(description='Update/add', button_style='success')
delete_option = Button(description='Delete', button_style='danger')

stats = HTML()

email_template = Textarea(value=default_email_template)
email_template.layout.width = '50%'
email_template.layout.height = '400px'
email_to = Text(description='Recipients', placeholder='tdee@wondr.ld, tdoo@wondr.ld')
include_student = Checkbox(description='send to student', value=False)
attach_pdf = Checkbox(description='Attach pdf', value=True)
send_email = Button(icon='fa-paper-plane', description='Send email', button_style='warning')

render_email = Button(description='Refresh', button_style='success')
email_result = HTML()
email_result.layout.width = '50%'

@render_email.on_click
def email_from_template(change=None):
    with orm.db_session():
        try:
            student_id = db.Submission[state.submission_id].student.id
        except AttributeError:
            email_result.value = ('<pre>Student id missing in submission, '
                                  'please provide it via validate_nr dashboard')
    email_result.value = ('<pre>' 
                          + emailing_feedback.form_email(state.exam_id, student_id,
                                                         template=email_template.value,
                                                         text_only=True)
                          + '</pre>')    

@send_email.on_click
def send_single_email(sender=None):
    recipients = list(map((lambda s: s.strip()), email_to.value.split(',')))
    with orm.db_session():
        try:
            student_id = db.Submission[state.submission_id].student.id
        except AttributeError:
            return  # No student ⇒ cannot form the email ⇒ do nothing.
    if include_student.value:
        with orm.db_session:
            # This may fail, should represent this in UI!
            recipients.append(db.Student[student_id].email)
    msg = emailing_feedback.form_email(state.exam_id, student_id, template=email_template.value,
                                       attach=attach_pdf.value, text_only=False)
    emailing_feedback.send([msg], recipients)


generate_report = Button(description='Exam statistics report')

@generate_report.on_click
def prepare_report_notebook(sender=None, target_folder='/node_modules/jupyter-dashboards-server/public'):
    if not os.path.exists(target_folder):
        target_folder = '.'
    with orm.db_session:
        target = db.Exam[state.exam_id].name + '_summary.html'

    jupyter = os.path.join(os.path.dirname(sys.executable), 'jupyter')
    subprocess.run('export EXAM_ID={};{} nbconvert --to html '
                   '--execute --HTMLExporter.exclude_input=True '
                   '--HTMLExporter.exclude_output_prompt=True --output {} summary_report.ipynb'
                   .format(state.exam_id, jupyter, os.path.join(target_folder, target)),
                   shell=True)
    alerts.value = '<script>window.open("/{0}", "_self")</script>'.format(target)

export_grades = Button(description='Export exam grades')

@export_grades.on_click
def export_exam_grade(sender=None, target_folder='/node_modules/jupyter-dashboards-server/public'):
    if not os.path.exists(target_folder):
        target_folder = '.'
    data = db.full_exam_data(state.exam_id)
    if export_format.value == 'DataFrame (detailed)':
        fname = 'scores_' + secrets.token_urlsafe(8) + '.pd'
        full_path = os.path.join(target_folder, fname)
        data.to_pickle(full_path)
    elif export_format.value == 'Spreadsheet (detailed)':
        fname = 'scores_' + secrets.token_urlsafe(8) + '.xlsx'
        full_path = os.path.join(target_folder, fname)
        data.to_excel(full_path)
    elif export_format.value == 'Spreadsheet (summary)':
        fname = 'scores_' + secrets.token_urlsafe(8) + '.xlsx'
        full_path = os.path.join(target_folder, fname)
        data = data.iloc[:, data.columns.get_level_values(1)=='total']
        data.columns = data.columns.get_level_values(0)
        data.to_excel(full_path)
    elif export_format.value == 'Complete database':
        fname = 'course_database_' + secrets.token_urlsafe(8) + '.sqlite'
        full_path = os.path.join(target_folder, fname)
        shutil.copy('course.sqlite', full_path)
    else:
        raise RuntimeError("Unrecognized export format")

    alerts.value = '<script>window.open("/{0}", "_self")</script>'.format(fname)
    subprocess.Popen('sleep 30; rm ' + full_path, shell=True)
    
export_format = Dropdown(
                    options=['Spreadsheet (detailed)',
                             'Spreadsheet (summary)',
                             'DataFrame (detailed)',
                             'Complete database',
                            ]
                )

confirm_full_feedback = Checkbox(description='Yes, email everyone', value=False)
email_full_feedback = Button(description='Email results to everyone', button_style='danger')
traitlets.directional_link((confirm_full_feedback, 'value'),
                           (email_full_feedback, 'disabled'),
                           operator.not_)
alerts = HTML()



@email_full_feedback.on_click
def email_all_feedback(sender=None):
    alerts.value = ("<script>alert('preparing and emailing all results"
                           " takes a lot of time. Do not leave the page while processing continues!')</script>")
    with orm.db_session:
        student_ids = list(db.Exam[state.exam_id].submissions.student.id)

    messages = [emailing_feedback.form_email(state.exam_id, sid, template=email_template.value,
                                             attach=attach_pdf.value, text_only=False)
                for sid in student_ids]
    failed = emailing_feedback.send(messages)
    if failed:
        msg = '<script>alert("Could not send emails to {}")</script>'
        alerts.value = msg.format(', '.join(str(student_ids[i]) for i in failed))
    else:
        alerts.value = '<script>alert("All emails sent successfully.")</script>'


def render_stats(change):
    if editing.selected_index != 2:
        return
    # Pandas is an overkill for merely making a table, but typing is hard.

    df = pandas.DataFrame({fo.text: (fo.solutions.count(), fo.score) 
                           for fo in db.Problem[state.problem_id].feedback_options}, 
                          index=['amount', 'score']).T.fillna(0).astype(int)
    df.index.name = "Feedback"
    stats.value = '<h4>Feedback frequencies and scores</h4>'
    stats.value += df._repr_html_()
    df = pandas.DataFrame(pandas.Series(Counter(sum(fo.score or 0 for fo in sol.feedback) 
                          for sol in db.Problem[state.problem_id].solutions)), columns=['Amount'])
    df.index.name = 'Score'
    stats.value += '<hr><h4>Score distribution</h4>'
    stats.value += df.T._repr_html_()
    stats.value += '<hr><h4>Feedback → student number</h4>'
    for fo in db.Problem[state.problem_id].feedback_options:
        students = filter((lambda s: s is not None), fo.solutions.submission.student.id)
        stats.value += '<p><h5>' + fo.text + '</h5>' + ', '.join(map(str, students)) + '</p>'

state.observe(render_stats, names=['exam_id', 'problem_id'])

def render_ui(_=None):
    image, grader = state.get_solution()
    answer.value = image
    render_grader(*grader)

def render_grader(graded_by, graded_at):
    if (graded_by is None) ^ (graded_at is None):
        raise ValueError('Invalid grading state')
    if graded_by is None:
        graded_label.value = ''
    else:
        graded_label.value = graded_label.template(graded_by, graded_at)
    problem_progress.value = state.num_graded()

state.observe(render_ui, names=['exam_id', 'submission_id', 'problem_id', 
                                'feedback_options', 'show_full_page'])

## Utilities

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

def load_image(path):
    with open(path, 'rb') as f:
        return f.read()    

## actual controllers - abusing decorator syntax FTW!
prev_stud.on_click(lambda _: state.previous_submission())
next_stud.on_click(lambda _: state.next_submission())
next_ungraded_stud.on_click(lambda _: state.next_ungraded())

@jump_to_stud.on_submit
def _(sender):
    try:
        student_id = int(sender.value.strip()[:7])
    except ValueError:
        return
    state.jump_to_student(student_id)
    
@observe(grader, names="value")
def _(sender):
    if sender['new'] == 'None':
        state.set_grader(None)
    else:   
        state.set_grader(sender['new'])

traitlets.link((options_feedback, 'value'), (state, 'selected_feedback'))
traitlets.link((remarks, 'value'), (state, 'remarks'))

## Controllers for the feedback options editing

save_option.on_click(state.commit_feedback_edit)
delete_option.on_click(state.delete_feedback_option)

In [ ]:
render_ui()

# Layout

In [ ]:
HBox([grader, jump_to_exam, jump_to_problem])

In [ ]:
# Main layout

grade = VBox([HBox([prev_stud, next_stud, next_ungraded_stud, jump_to_stud]),
              HBox([options_feedback, remarks]),
              HBox([problem_progress,  graded_label, full_page]), 
              answer])

edit_feedback = VBox([option_names, 
                      HBox([option_name, 
                            option_score,
                            save_option,
                            delete_option]),
                      
                      option_description])

individual_email = VBox([HBox([prev_stud, next_stud, jump_to_stud, render_email]),
                         HBox([email_template, email_result]),
                         HBox([email_to, include_student, attach_pdf, send_email]),
                        ])

summary = VBox([generate_report,
                HBox([export_grades, export_format]), 
                email_full_feedback, 
                confirm_full_feedback, 
                alerts])

editing = Tab(children=[grade, edit_feedback, stats, individual_email, summary])
editing.set_title(0, 'Grade')
editing.set_title(1, 'Edit feedback options')
editing.set_title(2, 'Problem statistics')
editing.set_title(3, 'individual email')
editing.set_title(4, 'summary & email')

editing.observe(render_stats, names=['selected_index'])

editing

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(jump_to_stud.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=state.students)))
display.display_javascript(prevent_unselect)

In [ ]:
import sys